# Intro to Using Rubin Data

## Learning Objectives

In this tutorial, you will learn:

  * How to work with an individual lightcurve
  * See examples of common filtering operations
  * How to calculate basic aggregation statistics

## Introduction

TODO: Add Introduction

# Loading Data

The details of loading Rubin data are discussed in [How to Access Data](TODO:linktorubin), so we'll just provide a starter codeblock below:

In [ ]:
from upath import UPath
import lsdb
from lsdb import ConeSearch

# This will eventually work
#base_path = UPath("/rubin/lincc_lsb_data") 
#object_collection = lsdb.read_hats(base_path / "object_collection_lite")

# In the meantime
# Cone search on ECDFS (Extended Chandra Deep Field South)
object_collection = lsdb.read_hats('/sdf/data/rubin/shared/lsdb_commissioning/hats/v29_0_0/dia_object_collection', search_filter=ConeSearch(ra=52.838, dec=-28.279, radius_arcsec=5000))
object_collection

As mentioned beneath the catalog dataframe, the view above is a "lazy" view of the data. Often, it's nice to preview the first few rows to better understand the contents of the dataset:

In [ ]:
object_collection.head(5)

## Example: Viewing a Single Lightcurve

In [ ]:
objectid = 609782208097419314
single_id = object_collection.id_search(values={"diaObjectId":objectid}).compute()
single_id


In [ ]:
from matplotlib.patches import Patch
import matplotlib.pyplot as plt
import numpy as np

first_lc = single_id.diaObjectForcedSource.iloc[0]

# Compute symmetric y-limits around 0 using 95% range
flux = first_lc['psfDiffFlux'].dropna()
limit = np.percentile(np.abs(flux), 97.5) + 100
y_min, y_max = -limit, limit

# Start plot
fig, ax = plt.subplots(2,1, figsize=(10, 10), dpi=200)

# Define band → color mapping
band_colors = {
    'u': 'blue',
    'g': 'green',
    'r': 'red',
    'i': 'orange',
    'z': 'purple',
    'y': 'brown'
}

# Plot each band with its color
for band, color in band_colors.items():
    band_data = first_lc[first_lc['band'] == band]
    if band_data.empty:
        continue
    ax[0].errorbar(
        band_data['midpointMjdTai'],
        band_data['psfDiffFlux'],
        yerr=band_data['psfDiffFluxErr'],
        fmt='o',
        color=color,
        ecolor=color,
        elinewidth=2,
        capsize=2,
        alpha=0.8,
        markeredgecolor='k',
        label=band
    )

    ax[1].errorbar(
        band_data['midpointMjdTai'],
        band_data['psfMag'],
        yerr=band_data['psfMagErr'],
        fmt='o',
        color=color,
        ecolor=color,
        elinewidth=2,
        capsize=2,
        alpha=0.8,
        markeredgecolor='k',
        label=band
    )

fig.suptitle(f'Object ID: {single_id["diaObjectId"].values[0]} RA: {single_id["ra"].values[0]:.5f}, Dec: {single_id["dec"].values[0]:.5f}')

ax[0].invert_yaxis()
ax[0].set_xlabel('MJD (midpointMjdTai)')
ax[0].set_ylabel('psfDiffFlux')
#ax[0].set_title(f'Object ID: {single_id["diaObjectId"].values[0]} RA: {single_id["ra"].values[0]:.5f}, Dec: {single_id["dec"].values[0]:.5f}', fontsize=12)
ax[0].set_ylim(y_min, y_max)
ax[0].set_xlim(60622, 60658)
ax[0].grid(True)
ax[0].legend(title='Band', loc='best')

ax[1].invert_yaxis()
ax[1].set_xlabel('MJD (midpointMjdTai)')
ax[1].set_ylabel('psfMag')
#ax[1].set_title(f'Object ID: {single_id["diaObjectId"].values[0]} RA: {single_id["ra"].values[0]:.5f}, Dec: {single_id["dec"].values[0]:.5f}', fontsize=12)
#ax[1].set_ylim(y_min, y_max)
ax[1].set_xlim(60622, 60658)
ax[1].grid(True)
ax[1].legend(title='Band', loc='best')
plt.tight_layout()

## Common Filtering Operations

### Filtering by Number of Sources

Provided the Source table(s) haven't been modified by any filtering operations, the "nDiaSources" column is directly provided and allows for easy filtering based on lightcurve length.

In [ ]:
oc_long_lcs = object_collection.query("nDiaSources > 10")
oc_long_lcs.head(5)

### Filtering by Photometric Band

Another common operation is to filter by band, which can done similarly to above, but using sub-column queries:

In [ ]:
oc_long_lcs = oc_long_lcs.query("diaObjectForcedSource.band == 'g'")
oc_long_lcs.head(5)

> **Note**: Filtering operations on "diaObjectForcedSource" are not propagated to "diaSource". Any filtering operations on "diaSource" should be applied in addition to any operations done on "diaObjectForcedSource".

### Filtering Empty Lightcurves

Sometimes, filters on lightcurves may throw out all observations for certain objects, leading to empty lightcurves as seen for one of the objects above. In this case, we can filter objects with empty lightcurves using `dropna`:

In [ ]:
oc_long_lcs = oc_long_lcs.dropna(subset="diaObjectForcedSource")
oc_long_lcs.head(5)

## Calculating Basic Statistics

While Rubin DP1 data has many statistics pre-computed in object table column, custom computation of statistics remains broadly useful.

### Calculating Mean Magnitude

In [ ]:
import numpy as np

def mean_mag(mag):
    return {"mean_psfMag": np.mean(mag)}

oc_mean_mags = oc_long_lcs.reduce(mean_mag, "diaObjectForcedSource.psfMag", meta={"mean_psfMag":np.float64})
oc_mean_mags.head(10)

We can add these statistics back to our Catalog via a `merge`:

In [ ]:
#oc_with_mag = oc_long_lcs.merge(oc_mean_mags, left_index=True, right_index=True)

In [ ]:
#oc_with_mag